<a href="https://colab.research.google.com/github/Yazeedx0/GenAI/blob/main/LoRA_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **First LoRA**

In [ ]:
!pip install torch
!pip install safetensors
!pip install transformers
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files='/content/Data-HU-23K.csv', split='train')

print(dataset[0])


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, PreTrainedTokenizerFast
from peft import get_peft_model, LoraConfig

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/arabian_gpt_trained-2")
tokenizer = PreTrainedTokenizerFast.from_pretrained("/content/drive/MyDrive/arabian_gpt_trained-2")

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(model, lora_config)

print(model)


In [ ]:
def preprocess_data(examples):
    questions = examples['Question']
    answers = examples['Answer']

    inputs = [question + " " + answer for question, answer in zip(questions, answers)]

    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512)
    model_inputs["labels"] = model_inputs["input_ids"]


    return model_inputs

train_dataset = dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/15135 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments, AdamW

training_args = TrainingArguments(
    output_dir="./results-3",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_strategy="epoch",
    fp16=True,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

trainer.train()

<ipython-input-8-a3dfad3fd4dc>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.098100
1000,0.097100


TrainOutput(global_step=1419, training_loss=0.09751295556141677, metrics={'train_runtime': 2238.151, 'train_samples_per_second': 20.287, 'train_steps_per_second': 0.634, 'total_flos': 1.190510026555392e+16, 'train_loss': 0.09751295556141677, 'epoch': 3.0})

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

save_path = '/content/drive/MyDrive/MyModels/Model-LoRA-3'

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("تم حفظ الموديل بنجاح على Google Drive في المسار:", save_path)


In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
import torch

model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/MyModels/Model-LoRA-3")
tokenizer = PreTrainedTokenizerFast.from_pretrained("/content/drive/MyDrive/MyModels/Model-LoRA-3")

tokenizer.pad_token = tokenizer.eos_token

def generate_text(prompt, temperature=1.0):
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, padding=True, max_length=512)

    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=150,
            temperature=temperature,
            top_k=50,
            top_p=0.95,
            do_sample=True
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if generated_text.lower().startswith(prompt.lower()):
        generated_text = generated_text[len(prompt):].strip()

    return generated_text

interface = gr.Interface(
    fn=generate_text,
    inputs=[gr.Textbox(label="Prompt"), gr.Slider(0, 2, step=0.1, value=1.0, label="Temperature")],
    outputs="text",
    title="اهلا بك في مساعد للجامعة الهاشمية تفضل!",
    description="أدخل نصًا للبدء بتوليد نصوص مشابهة بناءً على النموذج المدرب."
)

interface.launch()


# **second LoRA**

In [ ]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files='/content/processed_data.csv', split='train')

print(dataset[0])


Generating train split: 0 examples [00:00, ? examples/s]

{'السؤال': '   كيف يتم التحاق الموظف بالبرامج التدريبية؟', 'الجواب': 'يتم التحاقه بناءً على قرار من الرئيس، وتنسيب العميد أو المدير المعني، مع تحقيق الشروط المحددة ', 'الكلمة المفتاحية': 'البرامج التدريبية، التنسيب', 'التصنيف': 'قوانين العمل', 'score': '0.530917346'}


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, PreTrainedTokenizerFast
from peft import get_peft_model, LoraConfig

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/MyModels/Model-LoRA-3")
tokenizer = PreTrainedTokenizerFast.from_pretrained("/content/drive/MyDrive/MyModels/Model-LoRA-3")

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(model, lora_config)

print(model)

In [ ]:
def preprocess_data(examples):
    questions = examples['السؤال']
    answers = examples['الجواب']

    inputs = [question + " " + answer for question, answer in zip(questions, answers)]

    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512)
    model_inputs["labels"] = model_inputs["input_ids"]

    return model_inputs

train_dataset = dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/2223 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments, AdamW

training_args = TrainingArguments(
    output_dir="./results-3",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_strategy="epoch",
    fp16=True,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-61-617b15acd31f>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


TrainOutput(global_step=207, training_loss=0.21774336220561594, metrics={'train_runtime': 326.7733, 'train_samples_per_second': 20.409, 'train_steps_per_second': 0.633, 'total_flos': 1736275167019008.0, 'train_loss': 0.21774336220561594, 'epoch': 2.9784172661870505})

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

save_path = '/content/drive/MyDrive/MyModels/Model-LoRA-4'

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("تم حفظ الموديل بنجاح على Google Drive في المسار:", save_path)

Mounted at /content/drive


NameError: name 'model' is not defined

In [ ]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import gradio as gr
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
import torch

model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/MyModels/Model-LoRA-4")
tokenizer = PreTrainedTokenizerFast.from_pretrained("/content/drive/MyDrive/MyModels/Model-LoRA-4")

tokenizer.pad_token = tokenizer.eos_token

def generate_text(prompt, temperature=1.0):
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, padding=True, max_length=1024)

    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=100,
            temperature=temperature,
            top_k=50,
            top_p=0.95,
            do_sample=True
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if generated_text.lower().startswith(prompt.lower()):
        generated_text = generated_text[len(prompt):].strip()

    return generated_text

interface = gr.Interface(
    fn=generate_text,
    inputs=[gr.Textbox(label="Prompt"), gr.Slider(0, 2, step=0.1, value=1.0, label="Temperature")],
    outputs="text",
    title="اهلا بك في مساعد للجامعة الهاشمية تفضل!",
    description="أدخل نصًا للبدء بتوليد نصوص مشابهة بناءً على النموذج المدرب."
)

interface.launch()

ModuleNotFoundError: No module named 'gradio'

# **Thaird Training**

In [ ]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files='/content/Data-HU-23K.csv', split='train')

print(dataset[0])



{'Question': 'ما هو الحد الأعلى للعبء الدراسي في الفصل الصيفي الأول أو الثاني؟', 'Answer': '10 ساعات معتمدة، ولا يوجد حد أدنى.'}


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, PreTrainedTokenizerFast
from peft import get_peft_model, LoraConfig

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/MyModels/Model-LoRA-4")
tokenizer = PreTrainedTokenizerFast.from_pretrained("/content/drive/MyDrive/MyModels/Model-LoRA-4")

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(model, lora_config)

print(model)

In [ ]:
!pip install gradio

In [5]:
import gradio as gr
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
import torch

model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/arabian_gpt_fully_merged")
tokenizer = PreTrainedTokenizerFast.from_pretrained("/content/drive/MyDrive/arabian_gpt_fully_merged")

tokenizer.pad_token = tokenizer.eos_token

def generate_text(prompt, temperature=1.0):
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, padding=True, max_length=512)

    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=100,
            temperature=temperature,
            top_k=50,
            top_p=0.95,
            do_sample=True
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if generated_text.lower().startswith(prompt.lower()):
        generated_text = generated_text[len(prompt):].strip()

    return generated_text

interface = gr.Interface(
    fn=generate_text,
    inputs=[gr.Textbox(label="Prompt"), gr.Slider(0, 2, step=0.1, value=1.0, label="Temperature")],
    outputs="text",
    title="اهلا بك في مساعد للجامعة الهاشمية تفضل!",
    description="أدخل نصًا للبدء بتوليد نصوص مشابهة بناءً على النموذج المدرب."
)

interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6676c5493af86ca373.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# **Fourth Training**

In [ ]:
!pip install torch
!pip install safetensors
!pip install transformers
!pip install datasets

In [22]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files='/content/dataSEt_rows-2.csv', split='train')

print(dataset[1])


Generating train split: 0 examples [00:00, ? examples/s]

{'Question': '1 ما هي المادة التي تحدد أعداد الطلبة المقبولين في البرنامج الدولي', 'Answer': 'المادة 2'}


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, PreTrainedTokenizerFast
from peft import get_peft_model, LoraConfig

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/arabian_gpt_fully_merged")
tokenizer = PreTrainedTokenizerFast.from_pretrained("/content/drive/MyDrive/arabian_gpt_fully_merged")

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(model, lora_config)

print(model)


In [25]:
def preprocess_data(examples):
    questions = examples['Question']
    answers = examples['Answer']

    inputs = [question + " " + (answer if answer is not None else "لا إجابة") for question, answer in zip(questions, answers)]

    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512)

    model_inputs["labels"] = model_inputs["input_ids"]

    return model_inputs

train_dataset = dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/19479 [00:00<?, ? examples/s]

In [26]:
from transformers import Trainer, TrainingArguments, AdamW

training_args = TrainingArguments(
    output_dir="./results-3",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_strategy="epoch",
    fp16=True,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-26-a3dfad3fd4dc>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.183600
1000,0.165400
1500,0.158100


TrainOutput(global_step=1824, training_loss=0.16683941138418099, metrics={'train_runtime': 2776.8082, 'train_samples_per_second': 21.045, 'train_steps_per_second': 0.657, 'total_flos': 1.5303448565121024e+16, 'train_loss': 0.16683941138418099, 'epoch': 2.9963039014373716})

In [27]:
from google.colab import drive

drive.mount('/content/drive')

save_path = '/content/drive/MyDrive/arabian_gpt_fully_merged-2'

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("تم حفظ الموديل بنجاح على Google Drive في المسار:", save_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
تم حفظ الموديل بنجاح على Google Drive في المسار: /content/drive/MyDrive/arabian_gpt_fully_merged-2


In [41]:
import gradio as gr
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
import torch

model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/arabian_gpt_fully_merged-2")
tokenizer = PreTrainedTokenizerFast.from_pretrained("/content/drive/MyDrive/arabian_gpt_fully_merged-2")

tokenizer.pad_token = tokenizer.eos_token

def generate_text(prompt, temperature=1.0):
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, padding=True, max_length=512)

    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=150,
            temperature=temperature,
            top_k=0,
            top_p=0.90,
            do_sample=True
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if generated_text.lower().startswith(prompt.lower()):
        generated_text = generated_text[len(prompt):].strip()

    return generated_text

interface = gr.Interface(
    fn=generate_text,
    inputs=[gr.Textbox(label="Prompt"), gr.Slider(0, 2, step=0.1, value=1.0, label="Temperature")],
    outputs="text",
    title="اهلا بك في مساعد للجامعة الهاشمية تفضل!",
    description="أدخل نصًا للبدء بتوليد نصوص مشابهة بناءً على النموذج المدرب."
)

interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b08b2d7e4b7880bfb7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
